In [9]:
# small dataset is faster and allows easier comparison of clean and raw since we have for all data
import pandas as pd

copeland_filelist = "/usr/src/app/radtts/data/dataset-1674754128-thomas-copeland (3)/copeland/filelist.txt"


In [10]:
copeland = pd.read_csv(copeland_filelist, sep='|', index_col=None, header = None)

In [11]:
copeland[2] = 0

In [12]:
for i in range(copeland.shape[0]):
    copeland.iloc[i,0] = f"dataset-1674754128-thomas-copeland (3)/copeland/{copeland.iloc[i,0]}"

In [13]:
full_data = pd.concat([
                       copeland,
                      ])

In [14]:
import tempfile
def resample(in_path, output_path):

    # first read and write to fix codec issues
    rate, data = read(in_path)
    tf = tempfile.NamedTemporaryFile(suffix = '.wav')
    write(tf.name, rate = rate, data = np.asarray(data, dtype = np.int16))
    
    resampled_audio = librosa.load(tf.name)
    normalized_resampled_audio = np.asarray(
        resampled_audio[0]
        / np.abs(resampled_audio[0]).max()
        * MAX_WAV_VALUE
        / 2,
        dtype=np.int16,
    )
    write(
        data=normalized_resampled_audio,
        rate=resampled_audio[1],
        filename=output_path,
    )

In [16]:
import numpy as np
MAX_WAV_VALUE=32768
from scipy.io.wavfile import write, read
from tqdm import tqdm
import librosa
output_paths =[]
for rel_path in tqdm(full_data[0]):
    rel_uuid = rel_path.split('.wav')[-2]
    in_path = f"/usr/src/app/radtts/data/{rel_path}"
    output_path = f"/usr/src/app/radtts/data/{rel_uuid}_22k_normalized.wav"
    resample(in_path, output_path)
    output_paths.append(output_path)

100%|██████████| 103/103 [00:00<00:00, 357.34it/s]


In [17]:
full_data[0] = output_paths
full_data.to_csv('/usr/src/app/radtts/data/combined_030923_clean_small_22k_normalized.txt', sep = '|', index = None, header = None)